In [267]:
import json
import networkx as nx
import openai
from groq import Groq
from neo4j import GraphDatabase
import json
import json
import networkx as nx
import time

In [270]:
from neo4j import GraphDatabase

# Neo4j connection details
uri = "bolt://localhost:7687"  
username = "neo4j"                 
password = "123456789"         

class Neo4jSubgraphExtractor:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def get_subgraph(self):
        # Cypher query to get nodes User and Computer and relationships
        query = """
        MATCH (n)
        WHERE labels(n)[0] IN ['User', 'Computer']  
        OPTIONAL MATCH (n)-[r]->(m)
        WHERE labels(m)[0] IN ['User', 'Computer']  
        RETURN n, r, m
        """
        with self.driver.session() as session:
            result = session.run(query)
            nodes = set()
            edges = []
            for record in result:
                n = record["n"]
                m = record["m"]
                r = record["r"]
                if n:
                    nodes.add((n.id, dict(n)))  # Lấy ID và thuộc tính của node
                if m:
                    nodes.add((m.id, dict(m)))
                if r:
                    edges.append((r.start_node.id, r.end_node.id, dict(r)))  # Lấy ID và thuộc tính quan hệ
            return nodes, edges

# Tạo đối tượng kết nối và lấy subgraph
extractor = Neo4jSubgraphExtractor(uri, username, password)

# Truy vấn subgraph
nodes, edges = extractor.get_subgraph()

# Hiển thị kết quả
print("Nodes:")
for node in nodes:
    print(node)

print("\nEdges:")
for edge in edges:
    print(edge)

# Đóng kết nối
extractor.close()


/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_15270/2721252906.py:33: DeprecationWarning: `id` is deprecated, use `element_id` instead
  nodes.add((n.id, dict(n)))  # Lấy ID và thuộc tính của node


TypeError: unhashable type: 'dict'

In [271]:
class Neo4jSubgraphExtractor:
    def __init__(self, uri, username, password):
        self.driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        self.driver.close()

    def get_subgraph(self):
        query = """
        MATCH (n)-[r]->(m)
        WHERE labels(n)[0] IN ['User', 'Computer'] AND labels(m)[0] IN ['User', 'Computer']
        RETURN n, r, m
        """
        with self.driver.session() as session:
            result = session.run(query)
            
            # Kết quả sẽ được gom thành danh sách node và quan hệ
            nodes = {}
            edges = []

            for record in result:
                # Lấy node nguồn (n)
                n = record["n"]
                if n.id not in nodes:
                    nodes[n.id] = {
                        "identity": n.id,
                        "labels": list(n.labels),
                        "properties": dict(n),
                        "elementId": n.element_id
                    }

                # Lấy node đích (m)
                m = record["m"]
                if m.id not in nodes:
                    nodes[m.id] = {
                        "identity": m.id,
                        "labels": list(m.labels),
                        "properties": dict(m),
                        "elementId": m.element_id
                    }

                # Lấy thông tin quan hệ (r)
                r = record["r"]
                edges.append({
                    "identity": r.id,
                    "start": r.start_node.id,
                    "end": r.end_node.id,
                    "type": r.type,
                    "properties": dict(r),
                    "elementId": r.element_id
                })

            return list(nodes.values()), edges

# Khởi tạo và gọi hàm
extractor = Neo4jSubgraphExtractor(uri, username, password)
nodes, edges = extractor.get_subgraph()

# Hiển thị kết quả
import json
print("Nodes:")
print(json.dumps(nodes, indent=2))

print("\nEdges:")
print(json.dumps(edges, indent=2))

extractor.close()

Nodes:
[
  {
    "identity": 13,
    "labels": [
      "Computer"
    ],
    "properties": {
      "neo4jImportId": "13",
      "owned": false,
      "domain": "TestCompany.Local",
      "operatingsystem": "Windows 10",
      "name": "COMP00001.TestCompany.Local",
      "objectid": "S-1-5-21-883232822-274137685-4173207997-1000",
      "enabled": true
    },
    "elementId": "13"
  },
  {
    "identity": 316,
    "labels": [
      "User"
    ],
    "properties": {
      "neo4jImportId": "316",
      "owned": false,
      "displayname": "Hollis Camelo",
      "pwdlastset": 1632319288,
      "domain": "TestCompany.Local",
      "lastlogon": -1,
      "name": "HollisCamelo03@TestCompany.Local",
      "enabled": true,
      "objectid": "S-1-5-21-883232822-274137685-4173207997-1303"
    },
    "elementId": "316"
  },
  {
    "identity": 16,
    "labels": [
      "Computer"
    ],
    "properties": {
      "neo4jImportId": "16",
      "owned": false,
      "domain": "TestCompany.Local",
     

/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_15270/3163630826.py:24: DeprecationWarning: `id` is deprecated, use `element_id` instead
  if n.id not in nodes:
/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_15270/3163630826.py:26: DeprecationWarning: `id` is deprecated, use `element_id` instead
  "identity": n.id,
/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_15270/3163630826.py:25: DeprecationWarning: `id` is deprecated, use `element_id` instead
  nodes[n.id] = {
/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_15270/3163630826.py:34: DeprecationWarning: `id` is deprecated, use `element_id` instead
  if m.id not in nodes:
/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_15270/3163630826.py:36: DeprecationWarning: `id` is deprecated, use `element_id` instead
  "identity": m.id,
/var/folders/37/xpv32ytx11lbg70cx5xlqxr80000gn/T/ipykernel_15270/3163630826.py:35: DeprecationWarning: `id` is deprecated, use `element_id` instead
  node

In [29]:
# Thêm dữ liệu vào đồ thị
for entry in results:
    m = entry["m"]
    n = entry["n"]
    r_type, r_props = entry["r"]

    # Thêm các node
    G.add_node(m["id"], **m)
    G.add_node(n["id"], **n)

    # Thêm cạnh với properties
    G.add_edge(m["id"], n["id"], relationship_type=r_type, **r_props)

# Mã hóa dưới dạng incident encoder
print("Graph G has:")
for node in G.nodes(data=True):
    print(f"Node {node[0]} properties: {node[1]}")

print("\nIn the graph G:")
for edge in G.edges(data=True):
    node_a, node_b, props = edge
    print(f"Node {node_a} connects Node {node_b} with properties {props}")


TypeError: 'NoneType' object is not iterable

# GENERATING CYPHER QUERY

In [75]:
 # Function to generate cypher queries
def process_cypher_generation(prompt, model):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model= model,
    )

    return chat_completion.choices[0].message.content

In [76]:
client = Groq(
    api_key="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",
)

##  WWC-2019

In [265]:
prompt_orgi =  """

For each consistency rule that I provide, please generate a corresponding Cypher query to check the number of nodes or 
relationships that satisfy the rule. The rule could be related to node properties, relationship types, or dependencies between nodes. 
Provide the query in the format of valid Cypher syntax (important) and ready for execution in a Neo4j database. 
Below is the input data:
Graph Information:

- Nodes: Tournament, Team, Squad, Person, Match.
- Relationships: PLAYED_IN, NAMED, PARTICIPATED_IN, FOR, REPRESENTS, IN_SQUAD, SCORED_GOAL, COACH_FOR, IN_TOURNAMENT,
- Node properties: Tournament.name, Tournament.id, Tournament.shortName, Tournament.year, Team.name, Team.id, Squad.id, Person.id, Person.name, Person.dobm Match.id, Match.stage, Match.date


Expected Output:

The correct corresponding Cypher query to calculate the number of elements in the graph that satisfy a given rule (calculate support metric)"""


### Sliding window attention
### Zero-shot prompting

#### LLAMA

In [82]:
wwc_zero_llama =  """
- Rule 1: Unique Person ID - Description: Each person node should have a unique id property
- Rule 2: Person Node Properties - Description: Each person node should have a name and dob property
- Rule 3: Unique Match ID - Description: Each match node should have a unique id property
- Rule 4: Match Node Properties - Description: Each match node should have a date and stage property.
- Rule 5: Unique Squad ID - Description: Each squad node should have a unique id property
- Rule 6: Squad Node Properties - Description: Each squad node should have an id property
- Rule 7: Unique Team ID - Description: Each team node should have a unique id property
- Rule 8: Team Node Properties - Description: Each team node should have a name property
- Rule 9: Person-Match Relationship - Description: A person node can only be connected to a match node through a PLAYED_IN or SCORED_GOAL relationship
- Rule 10: Squad-Tournament Relationship - Description: A squad node can only be connected to a tournament node through a FOR relationship
- Rule 11: Team-Squad Relationship - Description: A team node can only be connected to a squad node through a NAMED relationship
- Rule 12: Team-Tournament Relationship
- Rule 13: Match-Tournament Relationship
"""

In [100]:
wwc_zero_llama_cypher = process_cypher_generation(wwc_zero_llama + prompt_orgi, "llama3-70b-8192")

In [101]:
print(wwc_zero_llama_cypher)

Here are the corresponding Cypher queries for each consistency rule:

**Rule 1: Unique Person ID**
```
MATCH (p:Person) WITH p.id as pid, COUNT(*) as count
WHERE count > 1
RETURN COUNT(*) as duplicate_person_ids
```
This query checks for duplicate `id` properties on `Person` nodes.

**Rule 2: Person Node Properties**
```
MATCH (p:Person) WHERE NOT exists(p.name) OR NOT exists(p.dob)
RETURN COUNT(*) as invalid_person_nodes
```
This query checks if all `Person` nodes have both `name` and `dob` properties.

**Rule 3: Unique Match ID**
```
MATCH (m:Match) WITH m.id as mid, COUNT(*) as count
WHERE count > 1
RETURN COUNT(*) as duplicate_match_ids
```
This query checks for duplicate `id` properties on `Match` nodes.

**Rule 4: Match Node Properties**
```
MATCH (m:Match) WHERE NOT exists(m.date) OR NOT exists(m.stage)
RETURN COUNT(*) as invalid_match_nodes
```
This query checks if all `Match` nodes have both `date` and `stage` properties.

**Rule 5: Unique Squad ID**
```
MATCH (s:Squad) WITH s

#### Mixtral 

In [108]:
wwc_zero_mixtral =  """
- Rule 1: Unique Tournament Identifier: Each tournament should have a unique identifier
- Rule 2: Unique Match Identifier within a Tournament: Each match should have a unique identifier within its respective tournament
- Rule 3: Unique Player Identifier
- Rule 4: Unique Squad within a Tournament: Each squad should have a unique identifier within its respective tournament. This rule ensures that there are no duplicate squad names in the same tournament
- Rule 5: Existence of Squad-Team Relationship: For any given tournament, every team should be associated with a squad
- Rule 6: Existence of Team-Match Relationships: A team should be involved in at least one match for each tournament they participate in
- Rule 7: Match Score Consistency: A match should have a score for both teams (no NULL values) if their match has a score defined
- Rule 8: Valid Player-Squad Relationship: A player should be associated with a squad, and that squad should belong to the tournament for which the player has played a match
- Rule 9: Valid Player-Match Goal Relationship: A player should score a goal in a match only if they played in that match
"""

In [119]:
wwc_zero_mixtral_cypher = process_cypher_generation(wwc_zero_mixtral + prompt_orgi, "llama3-70b-8192")

In [120]:
print(wwc_zero_mixtral_cypher)

Here are the Cypher queries to check the number of nodes or relationships that satisfy each consistency rule:

**Rule 1: Unique Tournament Identifier**
```
MATCH (t:Tournament)
WITH t.id AS id, COUNT(DISTINCT t.id) AS count
WHERE count = 1
RETURN COUNT(*) AS supported_tournaments
```
**Rule 2: Unique Match Identifier within a Tournament**
```
MATCH (t:Tournament)-[:IN_TOURNAMENT]->(m:Match)
WITH t.id AS tid, m.id AS mid, COUNT(DISTINCT m.id) AS count
WHERE count = 1
RETURN COUNT(*) AS supported_matches
```
**Rule 3: Unique Player Identifier**
```
MATCH (p:Person)
WITH p.id AS id, COUNT(DISTINCT p.id) AS count
WHERE count = 1
RETURN COUNT(*) AS supported_players
```
**Rule 4: Unique Squad within a Tournament**
```
MATCH (t:Tournament)-[:IN_TOURNAMENT]->(s:Squad)
WITH t.id AS tid, s.id AS sid, COUNT(DISTINCT s.id) AS count
WHERE count = 1
RETURN COUNT(*) AS supported_squads
```
**Rule 5: Existence of Squad-Team Relationship**
```
MATCH (s:Squad)-[:REPRESENTS]->(t:Team)
RETURN COUNT(*) AS

In [137]:
wwc_zero_mixtral_cypher = process_cypher_generation(wwc_zero_mixtral + prompt_orgi, "llama3-70b-8192")

In [138]:
print(wwc_zero_mixtral_cypher)

Here are the Cypher queries to check the number of nodes or relationships that satisfy each consistency rule:

**Rule 1: Unique Tournament Identifier**
```cypher
MATCH (t:Tournament) 
WITH t.id AS id, COUNT(*) AS count 
WHERE count > 1 
RETURN COUNT(*) AS num_duplicates
```
This query checks for duplicate `id` values across `Tournament` nodes. If any duplicates are found, it returns the count of duplicates.

**Rule 2: Unique Match Identifier within a Tournament**
```cypher
MATCH (t:Tournament)-[:IN_TOURNAMENT]-(m:Match) 
WITH t.id AS tournament_id, m.id AS match_id, COUNT(*) AS count 
WHERE count > 1 
RETURN COUNT(*) AS num_duplicates
```
This query checks for duplicate `id` values across `Match` nodes within the same `Tournament`. If any duplicates are found, it returns the count of duplicates.

**Rule 3: Unique Player Identifier**
```cypher
MATCH (p:Person) 
WITH p.id AS id, COUNT(*) AS count 
WHERE count > 1 
RETURN COUNT(*) AS num_duplicates
```
This query checks for duplicate `id`

In [163]:
wwc_zero_mixtral_cypher = process_cypher_generation(wwc_zero_mixtral + prompt_orgi, "mixtral-8x7b-32768")

In [164]:
print(wwc_zero_mixtral_cypher)

Rule 1: Unique Tournament Identifier
```cypher
MATCH (t:Tournament)
RETURN COUNT(DISTINCT t.id)
```

Rule 2: Unique Match Identifier within a Tournament
```cypher
MATCH (t:Tournament)-[:IN_TOURNAMENT]->(m:Match)
RETURN t.id, COUNT(DISTINCT m.id)
```

Rule 3: Unique Player Identifier
```cypher
MATCH (p:Person)
RETURN COUNT(DISTINCT p.id)
```

Rule 4: Unique Squad within a Tournament
```cypher
MATCH (t:Tournament)-[:IN_TOURNAMENT]->(s:Squad)
RETURN t.id, COUNT(DISTINCT s.id)
```

Rule 5: Existence of Squad-Team Relationship
```cypher
MATCH (t:Team)
RETURN COUNT(t) AS num_teams, SUM(CASE WHEN (t)-[:NAMED]->(:Squad) THEN 1 ELSE 0 END) AS num_named_squads
```

Rule 6: Existence of Team-Match Relationships
```cypher
MATCH (t:Team)-[:PARTICIPATED_IN]->(:Match)
RETURN COUNT(DISTINCT t.id)
```

Rule 7: Match Score Consistency
```cypher
MATCH (m:Match)
WHERE m.stage IS NOT NULL AND m.date IS NOT NULL
RETURN COUNT(m) AS num_matches, SUM(CASE WHEN m.homeTeamScore = -1 OR m.awayTeamScore = -1 THEN 

### Few-shot prompt 

#### LLAMA

In [175]:
wwc_few_llama = """
- Rule 1: Unique Match ID: Each Match node should have a unique id
- Rule 2: Valid Date Ranges: Match dates should fall within a valid range (e.g., between date(1900-01-01) and date(2030-12-31))
- Rule 3: Consistent Stage Values: Match stages should only have specific, pre-defined values (e.g., "Group Stage", "Quarter Finals", etc.)
- Rule 4: Valid Team Names: Team names should not be empty or null
- Rule 5: Unique Squad Names: Squad names should be unique within a tournament
- Rule 6: Person-Team Relationships: Each Person node should be associated with at most one Team node.
- Rule 7: Match-Team Relationships: Each Match node should be associated with exactly two Team nodes
- Rule 8: Valid Goal Scorers: A Person node can only score a goal in a Match node if they are part of the teams participating in that match
"""

In [178]:
wwc_few_llama_cypher = process_cypher_generation(wwc_few_llama + prompt_orgi, "llama3-70b-8192")

In [179]:
print(wwc_few_llama_cypher)

Here are the Cypher queries to check the number of nodes or relationships that satisfy each consistency rule:

**Rule 1: Unique Match ID**
```cypher
MATCH (m:Match)
WITH m.id AS id, COUNT(*) AS count
WHERE count > 1
RETURN COUNT(*) AS num_violations
```
This query checks for duplicate `id` values in the `Match` nodes. If there are any duplicates, it returns the count of violating matches.

**Rule 2: Valid Date Ranges**
```cypher
MATCH (m:Match)
WHERE m.date < date("1900-01-01") OR m.date > date("2030-12-31")
RETURN COUNT(*) AS num_violations
```
This query checks if the `date` property of `Match` nodes falls within the valid range. If any dates are outside the range, it returns the count of violating matches.

**Rule 3: Consistent Stage Values**
```cypher
MATCH (m:Match)
WHERE m.stage NOT IN ["Group Stage", "Quarter Finals", ...] // add all valid stage values here
RETURN COUNT(*) AS num_violations
```
This query checks if the `stage` property of `Match` nodes has only the specified val

In [180]:
wwc_few_mixtral = """
- Rule 1: Unique Match ID: Each Match node should have a unique id
- Rule 2: Match Date Uniqueness: Each Match node should have a unique date and stage within a specific tournament
- Rule 3: Unique Team Name: Each Team node should have a unique name within a specific tournament
- Rule 4: Unique Tournament Name: Each Tournament node should have a unique name
- Rule 5: Unique Squad: Each Squad node should have a unique id
- Rule 6: Person-Squad Relationship Consistency: Ensure that for each Person-Squad relationship, there is one and only one Squad that represents the Person
- Rule 7: Match-Team-Tournament Relationship Consistency: Ensure that for each Match-Team Relationship in a specific tournament, the Tournament node and Team node combinations are correct
- Rule 8: SCORED_GOAL Relationship Consistency: Ensure that no two SCORED_GOAL relationships between a Person and Match should have the same minute property
"""

In [217]:
wwc_few_mixtral_cypher = process_cypher_generation(wwc_few_mixtral + prompt_orgi + "pay attention calculate support metric", "llama3-70b-8192")

In [218]:
print(wwc_few_mixtral_cypher)

Here are the Cypher queries for each consistency rule:

**Rule 1: Unique Match ID**
```
MATCH (m:Match)
WITH m.id as id, COUNT(*) as count
WHERE count > 1
RETURN COUNT(*) as num_duplicates
```
This query groups matches by their `id` property and counts the number of duplicates. The `RETURN` clause gives the number of duplicates.

**Rule 2: Match Date Uniqueness**
```
MATCH (m:Match)
WITH m.date as date, m.stage as stage, m.tournament as tournament, COUNT(*) as count
WHERE count > 1
RETURN COUNT(*) as num_duplicates
```
This query groups matches by their `date`, `stage`, and `tournament` properties and counts the number of duplicates. The `RETURN` clause gives the number of duplicates.

**Rule 3: Unique Team Name**
```
MATCH (t:Team)
WITH t.name as name, t.tournament as tournament, COUNT(*) as count
WHERE count > 1
RETURN COUNT(*) as num_duplicates
```
This query groups teams by their `name` and `tournament` properties and counts the number of duplicates. The `RETURN` clause gives the n

## RAG 

### Zero-shot prompt

#### LLAMA

In [220]:
wwc_rag_zero_llama =  """
- Rule 1: Unique Node Representation: Each node should have a unique representation (id) within its respective category (e.g., player, team, coach)
- Rule 2: Consistent Node Labels: Each node should have a consistent label based on its category (e.g., player, team, coach).
- Rule 3: Valid Relationships Relationships between nodes should be valid and consistent, a player can only play for one team at a time, PERSON - COACH_FOR - Squad
- Rule 4: Valid Relationships Relationships between nodes should be valid and consistent,a coach can only coach one team at a time, Person - IN_SQUAD - Squad
- Rule 5: IN_SQUAD Relationship Consistency: The IN_SQUAD relationship should only exist between a player and a team.
- Rule 6: COACH_FOR Relationship Consistency: The COACH_FOR relationship should only exist between a coach and a team
- Rule 7: REPRESENTS Relationship Consistency: The REPRESENTS relationship should only exist between a player/coach and a team.
"""

In [225]:
wwc_rag_zero_llama_cypher = process_cypher_generation(wwc_rag_zero_llama + prompt_orgi + "pay attention calculate support metric", "llama3-70b-8192")

In [226]:
print(wwc_rag_zero_llama_cypher)

Here are the Cypher queries to check the number of nodes or relationships that satisfy each consistency rule:

**Rule 1: Unique Node Representation**
```cypher
MATCH (n) 
WHERE n.id IS NOT NULL 
WITH n, COUNT(DISTINCT n.id) AS id_count 
WHERE id_count > 1 
RETURN COUNT(*) AS inconsistencies;
```
This query checks for duplicate node IDs across all nodes in the graph.

**Rule 2: Consistent Node Labels**
```cypher
MATCH (n) 
WHERE NOT (n:Team OR n:Person OR n:Coach OR n:Tournament OR n:Squad OR n:Match) 
RETURN COUNT(*) AS inconsistencies;
```
This query checks for nodes without a consistent label (i.e., not one of the expected node types).

**Rule 3: Valid Relationships (PLAYER - IN_SQUAD - Squad)**
```cypher
MATCH (p:Person)-[:IN_SQUAD]-(s:Squad) 
WITH p, COUNT(DISTINCT s) AS squad_count 
WHERE squad_count > 1 
RETURN COUNT(*) AS inconsistencies;
```
This query checks for players who are part of more than one squad at the same time.

**Rule 4: Valid Relationships (COACH - COACH_FOR - Sq

In [262]:
wwc_rag_zero_mixtral = """
1. Unique Player Representation: Each player should be uniquely represented in the graph. This means that there should not be two nodes with the same id.
2. Unique Squad Membership: A player cannot be in two different squads at the same time. This means that there should not be two IN_SQUAD relationships for the same player (node with REPRESENTS label) and different squads (nodes with id starting with 11, 13, or 37)**
3. Each Tournament has a unique name: A tournament's name should be unique across all tournaments.
4. Unique Player-Squad Pairing: A player cannot be in the same squad more than once. This means that there should not be multiple IN_SQUAD relationships between the same player (node with REPRESENTS label) and the same squad (node with id starting with 11, 13, or 37)
5. Consistent Start and Substitution Minutes: If a player is substituted in a game, the minuteOn value in the PLAYED_IN relationship should be less than the minute value in the SCORED_GOAL relationship (if it exists).
6. Unique Goal Scoring: A player cannot score two goals in the same minute of the same game. This means that there should not be two SCORED_GOAL relationships for the same player (node with REPRESENTS label), the same game (node with id starting with 11, 13, or 37), and the same minute.
7. Consistent Coach-Team Pairing: A coach cannot coach two different teams at the same time. This means that there should not be two COACH_FOR relationships for the same coach (node with id starting with 12) and different teams (nodes with id starting with 11, 13, or 37).
"""

In [263]:
wwc_rag_zero_mixtral_cypher = process_cypher_generation(wwc_rag_zero_mixtral + prompt_orgi , "llama3-70b-8192")

In [264]:
print(wwc_rag_zero_mixtral_cypher)

Here are the Cypher queries to check the number of nodes or relationships that satisfy each consistency rule:

**Rule 1: Unique Player Representation**
```cypher
MATCH (p:Person)
WITH p.id as pid, COUNT(DISTINCT p) as count
WHERE count > 1
RETURN COUNT(*) as duplicate_players
```
This query finds all `Person` nodes with duplicate `id` properties and returns the count of those duplicates.

**Rule 2: Unique Squad Membership**
```cypher
MATCH (p:Person)-[:IN_SQUAD]->(s:Squad)
WITH p.id as pid, s.id as sid, COUNT(DISTINCT s) as count
WHERE count > 1
RETURN COUNT(*) as multiple_squad_memberships
```
This query finds all `Person` nodes with multiple `IN_SQUAD` relationships to different `Squad` nodes and returns the count of those duplicates.

**Rule 3: Each Tournament has a unique name**
```cypher
MATCH (t:Tournament)
WITH t.name as tname, COUNT(DISTINCT t) as count
WHERE count > 1
RETURN COUNT(*) as duplicate_tournament_names
```
This query finds all `Tournament` nodes with duplicate `name

## CYBERSECURITY

### Sliding window attention

### Zero-shot prompt

#### LLAMA

In [266]:
cyber_win_zero_llama = """
- Rule 1: If a node has the label Computer, it must have the properties operatingsystem, name, objectid, and enabled
- Rule 2: If a node has the label User, it must have the properties displayname, pwdlastset, domain, lastlogon, name, objectid, and enabled
- Rule 3: If a relationship has the type HAS_SESSION, it must involve a Computer node and a User node.
- Rule 4: If a relationship has the type MEMBER_OF, it must connect a User node to a Group node or a Domain node (excluding relationships between Domain nodes and OU nodes).
- Rule 5:If a relationship has the type ADMIN_TO, it must connect a User node or a Group node to a Domain node or a Computer node
- Rule 6: For each Domain node, there must be at most one Domain node connected via the PARENT_OF relationship
- Rule 7: For each Domain node, there must be at least one Domain node connected via the SUB_DOMAINS relationship
- Rule 8: For each Group node, there must be at least one User node connected via the MEMBER_OF relationship
- Rule 9: For each Domain node, there must be at least one OU node connected via the CONTAINS relationship
- Rule 10: For each GPO node, there must be exactly one OU node connected via the AFFECTS relationship

"""